<a href="https://colab.research.google.com/github/stick35em10/Teste-A-B/blob/main/notebooks/teste_AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalations
#!pip install -r requirementes.txt
#!pip install pandas

In [2]:
# instalations
#!pip install statsmodels

# **0.0. Import packages**

In [3]:
# imports
import pandas as pd
#from google.colab import drive


In [4]:
#import statsmodels.stats.power as sms
import math

# 0.1 Help Functions

In [19]:
# =====================
# 1. TESTE Z PARA PROPORÇÕES
# =====================

def realizar_teste_ab(n_control, n_treatment, conv_control, conv_treatment, alpha=0.05):
    """
    Realiza teste A/B para diferença de proporções
    """
    print("🎯 TESTE A/B - DIFERENÇA DE PROPORÇÕES")
    print("=" * 50)

    # Calcular número de conversões
    conversoes_control = int(n_control * conv_control)
    conversoes_treatment = int(n_treatment * conv_treatment)

    print(f"📊 Dados do Experimento:")
    print(f"   Grupo Controle: {conversoes_control}/{n_control} = {conv_control:.3%}")
    print(f"   Grupo Tratamento: {conversoes_treatment}/{n_treatment} = {conv_treatment:.3%}")
    print(f"   Diferença absoluta: {(conv_treatment - conv_control):.3%}")
    print(f"   Melhoria relativa: {((conv_treatment/conv_control)-1):.2%}")
    print()

    # Executar teste Z para duas proporções
    count = np.array([conversoes_control, conversoes_treatment])
    nobs = np.array([n_control, n_treatment])

    # Teste bicaudal (alternative='two-sided')
    z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')

    print("📈 RESULTADOS DO TESTE ESTATÍSTICO")
    print("=" * 50)
    print(f"   Estatística Z: {z_stat:.4f}")
    print(f"   Valor-p: {p_value:.6f}")
    print(f"   Nível de significância (α): {alpha}")
    print()

    # Interpretação do resultado
    if p_value < alpha:
        print("✅ RESULTADO: SIGNIFICATIVO")
        print("   Rejeitamos a hipótese nula (H₀)")
        print("   A diferença é estatisticamente significativa!")
    else:
        print("❌ RESULTADO: NÃO SIGNIFICATIVO")
        print("   Não rejeitamos a hipótese nula (H₀)")
        print("   A diferença pode ser devido ao acaso")

    return z_stat, p_value

# =====================
# 2. INTERVALO DE CONFIANÇA
# =====================

def calcular_intervalo_confianca(n_control, n_treatment, conv_control, conv_treatment, conf_level=0.95):
    """
    Calcula o intervalo de confiança para a diferença
    """
    print("\n🎯 INTERVALO DE CONFIANÇA")
    print("=" * 50)

    # Diferença observada
    diff = conv_treatment - conv_control

    # Erro padrão da diferença
    se_control = np.sqrt(conv_control * (1 - conv_control) / n_control)
    se_treatment = np.sqrt(conv_treatment * (1 - conv_treatment) / n_treatment)
    se_diff = np.sqrt(se_control**2 + se_treatment**2)

    # Valor Z para o nível de confiança
    z_value = stats.norm.ppf(1 - (1 - conf_level) / 2)

    # Intervalo de confiança
    margem_erro = z_value * se_diff
    ic_inferior = diff - margem_erro
    ic_superior = diff + margem_erro

    print(f"   Diferença observada: {diff:.4f} ({diff:.3%})")
    print(f"   Erro padrão: {se_diff:.6f}")
    print(f"   Intervalo de {conf_level:.0%} confiança:")
    print(f"   [{ic_inferior:.4f}, {ic_superior:.4f}]")
    print(f"   Ou: [{ic_inferior:.3%}, {ic_superior:.3%}]")

    return (ic_inferior, ic_superior)

# =====================
# 3. PODER ESTATÍSTICO
# =====================

def analisar_poder_estatistico(n_control, n_treatment, conv_control, conv_treatment, alpha=0.05):
    """
    Analisa o poder estatístico do teste
    """
    print("\n🎯 ANÁLISE DO PODER ESTATÍSTICO")
    print("=" * 50)

    # Tamanho do efeito
    effect_size = proportion_effectsize(conv_control, conv_treatment)

    # Calcular poder estatístico
    power_analysis = NormalIndPower()
    poder_calculado = power_analysis.solve_power(
        effect_size=effect_size,
        nobs1=n_control,
        alpha=alpha,
        power=None
    )

    print(f"   Tamanho do efeito: {effect_size:.4f}")
    print(f"   Poder estatístico calculado: {poder_calculado:.3f}")

    # Verificar se o poder é adequado
    if poder_calculado >= 0.8:
        print("   ✅ Poder estatístico adequado (≥ 0.8)")
    else:
        print("   ⚠️  Poder estatístico insuficiente (< 0.8)")

    return poder_calculado

# =====================
# 4. TAMANHO MÍNIMO DETECTÁVEL (MDE)
# =====================

def calcular_mde(n_control, n_treatment, conv_control, alpha=0.05, power=0.80):
    """
    Calcula o efeito mínimo detectável (MDE)
    """
    print("\n🎯 EFEITO MÍNIMO DETECTÁVEL (MDE)")
    print("=" * 50)

    power_analysis = NormalIndPower()

    # Calcular MDE para o poder desejado
    effect_size_required = power_analysis.solve_power(
        nobs1=n_control,
        alpha=alpha,
        power=power,
        effect_size=None
    )

    # Converter effect size de volta para diferença de proporções
    # Para proporções: effect_size = 2 * arcsin(sqrt(p1)) - 2 * arcsin(sqrt(p2))
    # Podemos aproximar invertendo a fórmula
    from math import sin, asin, sqrt

    # Aproximação do MDE
    arcsin_p1 = 2 * asin(sqrt(conv_control))
    arcsin_p2_required = arcsin_p1 + effect_size_required
    p2_required = (sin(arcsin_p2_required / 2)) ** 2
    mde_absoluto = p2_required - conv_control

    print(f"   Conversão base (controle): {conv_control:.3%}")
    print(f"   MDE absoluto: {mde_absoluto:.4f} ({mde_absoluto:.3%})")
    print(f"   Efeito observado: {conversion_treatment - conversion_control:.4f} ({(conversion_treatment - conversion_control):.3%})")

    # Comparar com efeito observado
    efeito_observado = conversion_treatment - conversion_control
    if efeito_observado >= mde_absoluto:
        print("   ✅ Efeito observado maior que MDE")
    else:
        print("   ⚠️  Efeito observado menor que MDE")

    return mde_absoluto

# =====================
# 5. EXECUÇÃO COMPLETA
# =====================

def executar_analise_completa():
    """
    Executa toda a análise estatística do teste A/B
    """
    print("🧪 ANÁLISE ESTATÍSTICA COMPLETA DO TESTE A/B")
    print("=" * 60)

    # 1. Teste de hipóteses
    z_stat, p_value = realizar_teste_ab(
        n_control, n_treatment,
        conversion_control, conversion_treatment,
        alpha
    )

    # 2. Intervalo de confiança
    ic = calcular_intervalo_confianca(
        n_control, n_treatment,
        conversion_control, conversion_treatment
    )

    # 3. Poder estatístico
    poder = analisar_poder_estatistico(
        n_control, n_treatment,
        conversion_control, conversion_treatment,
        alpha
    )

    # 4. MDE
    mde = calcular_mde(n_control, n_treatment, conversion_control, alpha, power)

    # 5. Resumo executivo
    print("\n" + "🎯 RESUMO EXECUTIVO" + "=" * 50)

    efeito_observado = conversion_treatment - conversion_control
    significativo = p_value < alpha
    poder_adequado = poder >= 0.8

    print(f"📊 Resultado Principal:")
    print(f"   • Diferença: {efeito_observado:.3%} ({conversion_treatment/conversion_control-1:+.1%})")
    print(f"   • Significativo: {'✅ SIM' if significativo else '❌ NÃO'}")
    print(f"   • Poder adequado: {'✅ SIM' if poder_adequado else '❌ NÃO'}")
    print(f"   • Valor-p: {p_value:.6f}")

    print(f"\n💡 Recomendação:")
    if significativo and poder_adequado:
        print("   ✅ IMPLEMENTAR - Resultado confiável e significativo")
    elif significativo and not poder_adequado:
        print("   ⚠️  CAUTELA - Significativo mas poder insuficiente")
    elif not significativo and poder_adequado:
        print("   ❌ NÃO IMPLEMENTAR - Diferença não significativa")
    else:
        print("   🔄 MAIS DADOS - Poder insuficiente para conclusão")

    return {
        'z_statistic': z_stat,
        'p_value': p_value,
        'confidence_interval': ic,
        'power': poder,
        'mde': mde,
        'significant': significativo,
        'adequate_power': poder_adequado
    }


# **1.0. Load data**

In [5]:
#!pwd
#!ls /content/drive/MyDrive/'Colab Notebooks'/'Teste AB'/dataset/ab_data.crdownload #/sample_data/
#df_raw.shape: (18583, 5)
#path_ab_data = '/content/drive/MyDrive/Colab Notebooks/Teste AB/dataset/ab_data.crdownload' #/sample_data/'
#df_raw.shape: (294478, 5)
#path_ab_data = '/content/drive/MyDrive/Colab Notebooks/Teste AB/dataset/ab_data_.csv'
#!ls ../data/raw/ab_data_.csv
path_ab_data ='../data/raw/ab_data_.csv' #data/raw/ab_data_.csv'



In [6]:
df_raw=pd.read_csv(path_ab_data) #'/content/drive/MyDrive/Colab Notebooks/Teste AB/dataset/ab_data.crdownload')
df_raw.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [7]:
df_raw.shape

(294478, 5)

# **2 Design de Experimentos**

### **2.1. Formulação de Hipóteses**

# 🧪 Passo 1: Definir o Problema de Negócio 
## 🎯 Objetivo 
Determinar se a nova página (new_page) resulta em uma taxa de conversão maior do que a página antiga (old_page).

### 2.1.1 Definição da hipótese nula

H₀ (Hipótese Nula): A nova página não aumenta a taxa de conversão.

### 2.1.2 Definição da hipótese alternativa

H₁ (Hipótese Alternativa): A nova página aumenta a taxa de conversão.

#### 2.1.3 Escolha do tipo de teste:Uma ou duas caudas

#### 2.1.4 Definição do nível de confiança do experimento.()Escolha do tipo de teste:Uma ou duas caudas

### **2.2. Parâmetros do Experimento**

In [8]:
# nivel de confianca
confidence_level    = 0.95

# nivel de significancia
significance_level  = 0.05

# conversao da pagina actual e da nova pagina
p1 = 0.13 # medido
p2 = 0.15 # esperado pelo time de negocio

In [9]:
# tanho do efeito
# AttributeError: module 'statsmodels.stats.power' has no attribute 'proportion_effectsize'
#ffect_size = sms.proportion_effectsize(p1, p2) #AttributeError: module 'statsmodels.stats.power' has no attribute 'proportion_effectsize'

# The function 'proportion_effectsize' is deprecated.
# Instead, use 'compute_effect_size' from 'statsmodels.stats.proportion'
from statsmodels.stats.proportion import proportion_effectsize
effect_size = proportion_effectsize(p1, p2)
print(f"O tamanho do efeito é {effect_size}")
# 


O tamanho do efeito é -0.0576728617308947


In [10]:
import numpy as np

# effect_size = sms.proportion_effectsize(p1, p2) #This function might be deprecated
# Instead, try using the following:
from statsmodels.stats.power import TTestIndPower, NormalIndPower
#effect_size = (p2 - p1) / np.sqrt(p1 * (1 - p1)) # Calculate effect size manually

# Poder estatística
power = 0.80 # erro de oportunidade

# sample size
#AttributeError: 'NormalIndPower' object has no attribute 'solver_power'
#sample_n = sms.NormalIndPower().solver_power(
sample_n = NormalIndPower().solve_power(
    effect_size=effect_size,
    power=power,
    alpha=significance_level
) #4720
#nobs1=None,
#ratio=1 # Assuming equal sample sizes for both groups

sample_n = math.ceil(sample_n)
#sample_n # amostras para cada um dos grupos
print(f' {sample_n} amostras para cada um dos grupos, de controle e de tratamento')

 4720 amostras para cada um dos grupos, de controle e de tratamento


# 3.0 Colecta dos dados

## 3.1.Análise descritiva dos dados

In [11]:
# 2.0 Análise descritiva dos dados
print(f' A base de dados apresenta {df_raw.shape[0]} rows and {df_raw.shape[1]} columns ')
#print(f'Number of ')
#print(f'Columns: {df_raw.columns.to_list()}

 A base de dados apresenta 294478 rows and 5 columns 


## 3.2.Verificação dos dados faltantes, check NA

In [12]:
# Interpretação do resultado
if df_raw.isna().sum().sum()==0:
    print("✅ Os dados não apresentam dados faltantes")
    print(" Podemos proseguir")
    #print("   A diferença é estatisticamente significativa!")
    #elif : 
else:
    print(f'❌ Os dados tem dados faltantes, um total de {df_raw.isna().sum().sum()}')
    print(" Devemos analisar antes de prosseguir")
    print({df_raw.isna().sum()})
#print("   A diferença pode ser devido ao acaso")

✅ Os dados não apresentam dados faltantes
 Podemos proseguir


## 3.2.Conferir as "flags"

In [13]:
# duplicado
#len(
df_raw[['user_id','group','landing_page']].groupby(['group','landing_page']).count().reset_index() #.query('group == 2 and landing_page == 2')#)

,group,landing_page,user_id
0,control,new_page,1928
1,control,old_page,145274
2,treatment,new_page,145311
3,treatment,old_page,1965


In [14]:
#len(
df_raw[['user_id','group','landing_page']].groupby(['group','landing_page']).count().reset_index() #.query('group == 2 and landing_page == 2')#)

,group,landing_page,user_id
0,control,new_page,1928
1,control,old_page,145274
2,treatment,new_page,145311
3,treatment,old_page,1965


In [15]:
#df_raw[['user_id','group','landing_page']].groupby(['user_id']).count().reset_index().query("group == 'control' and (landing_page == 'new_page' or landing_page == 'old_page')")
linhas_com_duplicação=len(df_raw[['user_id','group','landing_page']].groupby(['user_id']).count().reset_index().query('landing_page >1'))
percentagem_duplicado=linhas_com_duplicação/df_raw.shape[0]

# Interpretação do resultado
if percentagem_duplicado < 0.05:
    print(f'Dados iniciais {df_raw.shape}')
    print()
    print(f'✅ A percentagem dos dados duplicados são inferiores que 5%, isto é com {round(percentagem_duplicado*100,2)} % de linhas duplicadas')
    print('portando são considerados poucos dados duplicados e podemos deletar os dados duplicados ')
    df_user_delete = df_raw[['user_id','group','landing_page']].groupby(['user_id']).count().reset_index().query('landing_page >1')['user_id']
    #data without duplicated
    #print("data without duplicated")
    print()
    df1 = df_raw[~df_raw['user_id'].isin(df_user_delete)] # .groupby(['user_id']).count().reset_index().query('landing_page >1')
    #df1.head()
    # todo: faser uma função para verificar se existe dados duplicados, para usarmos e voltar a chamar para confirmar se ficou algum dado duplicado
    print(f'Total de dados apagagos {df_raw.shape[0]-df1.shape[0]}')
    print()
    print(df1.head())
    print()
    #print(df1.shape)
    print(f'Dados finais {df1.shape}')
    #print(" Podemos proseguir")
    #print("   A diferença é estatisticamente significativa!")
    #elif : 
else:
    print(f'❌ A percentagem dos dados duplicados é superior que 5%, com {round(percentagem_duplicado*100,2)} % de linhas duplicadas')
    print('portando são considerados muitos dados duplicados e devemos tratar os dados duplicados ')
    #print(f' Os dados tem dados faltantes, um total de {df_raw.isna().sum().sum()}')
    #print({df_raw.isna().sum()})
#print("   A diferença pode ser devido ao acaso")
#print(round(percentagem_duplicado*100,2), '% de linhas duplicadas')

#print('portando são poucos dados duplicados poís é menor que 5%, portanto podemos deletar os duplicados ')

#df_raw[['user_id','group','landing_page']].reset_index().query('user_id in [630052,630126,630137]') #43,111,122]

Dados iniciais (294478, 5)

✅ A percentagem dos dados duplicados são inferiores que 5%, isto é com 1.32 % de linhas duplicadas
portando são considerados poucos dados duplicados e podemos deletar os dados duplicados 

Total de dados apagagos 7788

   user_id                   timestamp      group landing_page  converted
0   851104  2017-01-21 22:11:48.556739    control     old_page          0
1   804228  2017-01-12 08:01:45.159739    control     old_page          0
2   661590  2017-01-11 16:55:06.154213  treatment     new_page          0
3   853541  2017-01-08 18:28:03.143765  treatment     new_page          0
4   864975  2017-01-21 01:52:26.210827    control     old_page          1

Dados finais (286690, 5)


### 2.2.Amostragem aleatoria dos grupos Controle e Tratamento

In [16]:
### 2.2.Amostragem aleatoria dos grupos Controle e Tratamento
# Control Group
df_control_sample = df1[df1['group'] == 'control'].sample(n=sample_n, random_state=42)
print('Size of Control or old_page Group: {}'.format(df_control_sample.shape[0]))

# Treatment Group
df_treatment_sample = df1[df1['group'] == 'treatment'].sample(n=sample_n, random_state=42)
print('Size of Treatment or new_page Group: {}'.format(df_treatment_sample.shape[0]))

# Total sample Size
df_ad = pd.concat([df_control_sample, df_treatment_sample], axis=0)
print("Total sample Size",df_ad.shape[0])

#df_ad_= pd.concat([df_control_sample, df_treatment_sample]).reset_index(drop=True) #res.reindex(df_raw.index)
#df_ad_.head()
#print(df_ad_.shape)
#
# Treatement Group
# Total Sample sizze

Size of Control or old_page Group: 4720
Size of Treatment or new_page Group: 4720
Total sample Size 9440


In [17]:
df_raw.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


### 2.3 Calculo da métrica de intersse entre os Grupos (conversão de cada página)

In [18]:
### 2.3 Calculo da métrica de intersse entre os Grupos (conversão de cada página)
# ============================= Control Group =============================
#sales = df_control_sample.loc[df_control_sample['landing_page'] == 'old_page', 'converted'].sum()
sales = df_control_sample.loc[df_control_sample['converted'] == 1, 'converted'].sum()
visit = len(df_control_sample)

conversion_rate_control = sales / visit
print('Conversion Rate of Control Group: {}'.format(conversion_rate_control))

# ============================= Treatment Group =======================
sales = df_treatment_sample.loc[df_treatment_sample['converted'] == 1, 'converted'].sum()
visit = len(df_treatment_sample)

conversion_rate_treatment = sales / visit
print('Conversion Rate of Treatment Group: {}'.format(conversion_rate_treatment))

# conver
visit


Conversion Rate of Control Group: 0.11546610169491525
Conversion Rate of Treatment Group: 0.12902542372881357


4720

# 4.0 Testando as hipóteses

In [27]:
n_control=visit
n_treatment=visit
conversion_control=conversion_rate_control
conversion_treatment=conversion_rate_treatment
alpha=significance_level



# Importações necessárias - VERSÃO CORRIGIDA
#import numpy as np
#import pandas as pd
from scipy import stats
#import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest#, proportion_effectsize
#from statsmodels.stats.power import NormalIndPower
#import math
executar_analise_completa()

🧪 ANÁLISE ESTATÍSTICA COMPLETA DO TESTE A/B
🎯 TESTE A/B - DIFERENÇA DE PROPORÇÕES
📊 Dados do Experimento:
   Grupo Controle: 545/4720 = 11.547%
   Grupo Tratamento: 609/4720 = 12.903%
   Diferença absoluta: 1.356%
   Melhoria relativa: 11.74%

📈 RESULTADOS DO TESTE ESTATÍSTICO
   Estatística Z: -2.0109
   Valor-p: 0.044336
   Nível de significância (α): 0.05

✅ RESULTADO: SIGNIFICATIVO
   Rejeitamos a hipótese nula (H₀)
   A diferença é estatisticamente significativa!

🎯 INTERVALO DE CONFIANÇA
   Diferença observada: 0.0136 (1.356%)
   Erro padrão: 0.006741
   Intervalo de 95% confiança:
   [0.0003, 0.0268]
   Ou: [0.035%, 2.677%]

🎯 ANÁLISE DO PODER ESTATÍSTICO
   Tamanho do efeito: -0.0414
   Poder estatístico calculado: 0.521
   ⚠️  Poder estatístico insuficiente (< 0.8)

🎯 EFEITO MÍNIMO DETECTÁVEL (MDE)
   Conversão base (controle): 11.547%
   MDE absoluto: 0.0191 (1.906%)
   Efeito observado: 0.0136 (1.356%)
   ⚠️  Efeito observado menor que MDE

🎯 RESUMO EXECUTIVO================

{'z_statistic': -2.0109005932696107,
 'p_value': 0.044335957690484505,
 'confidence_interval': (0.00034629185460578575, 0.02677235221319084),
 'power': 0.5206334688001045,
 'mde': 0.01905933473393359,
 'significant': True,
 'adequate_power': False}

# 5.0 Conclusão e Considerações Financeiras

## ✅ Resultado
A nova página apresentou uma taxa de conversão maior (12,90% vs 11,55%). A diferença é estatisticamente significativa ao nível de 5%.

## 💡 Impacto Financeiro
### Suponha que:

Tráfego mensal = 100.000 usuários

Valor médio por conversão = R$ 50

Melhoria esperada:

Conversões adicionais = (0,1290 - 0,1155) × 100.000 ≈ 1.350

### Ganho mensal = 1.350 × R$ 50 = R$ 67.500

# 🚀 Recomendação
Implementar a nova página devido:
* ao aumento significativo na taxa de conversão e 
* retorno financeiro positivo.
